# **AI for DDoS Detection**
##                  Kanagha Santhoshini (19BAI1044)
##                  Arvindh A R (19BAI1106)
##                  Manasa Y K (19BAI1108)

In [ ]:
!pip install kamene

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Bidirectional
import pandas as pd

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
datasetname='CICDS_Wednesday.csv'

In [ ]:
df = pd.read_csv(datasetname)
df.drop(['Flow ID'],axis=1,inplace=True)
df

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Label
0,192.168.10.14,49459,209.48.71.168,80,6,5/7/2017 8:42,BENIGN
1,192.168.10.17,49453,192.168.10.3,389,6,5/7/2017 8:42,BENIGN
2,192.168.10.17,46124,192.168.10.3,88,6,5/7/2017 8:42,BENIGN
3,192.168.10.17,49454,192.168.10.3,389,6,5/7/2017 8:42,BENIGN
4,192.168.10.17,46126,192.168.10.3,88,6,5/7/2017 8:42,BENIGN
...,...,...,...,...,...,...,...
692698,192.168.10.14,51114,192.168.10.3,53,17,5/7/2017 12:10,BENIGN
692699,192.168.10.16,24054,192.168.10.3,53,17,5/7/2017 3:02,BENIGN
692700,23.208.163.130,443,192.168.10.51,58030,6,5/7/2017 10:06,BENIGN
692701,192.168.10.14,51694,192.168.10.3,53,17,5/7/2017 1:19,BENIGN


In [ ]:
for i in range(692703):
  c = df[' Timestamp'].iloc[i].split(' ')
  time = c[1].split(':')
  time = int(time[0])*100+int(time[1])
  df[' Timestamp'].iloc[i]=time

In [ ]:
df.columns

In [ ]:
df

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Label
0,152.163.64.2,443,192.168.10.16,58894,6,100,BENIGN
1,172.217.10.40,443,192.168.10.5,51906,6,100,BENIGN
2,192.168.10.5,51942,199.59.148.12,443,6,100,BENIGN
3,50.97.40.233,443,192.168.10.5,51902,6,100,BENIGN
4,192.168.10.16,35235,192.168.10.3,53,17,100,BENIGN
...,...,...,...,...,...,...,...
692623,192.168.10.16,30134,192.168.10.3,53,17,1259,BENIGN
692624,192.168.10.16,60478,54.230.38.82,443,6,1259,BENIGN
692625,172.217.3.110,80,192.168.10.16,38098,6,1259,BENIGN
692626,192.168.10.16,39020,172.217.10.238,443,6,1259,BENIGN


In [ ]:
df[' Label'].value_counts()

BENIGN              439958
DoS Hulk            231073
DoS GoldenEye        10291
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name:  Label, dtype: int64

In [ ]:
def count_occurrences(cur_index, lookback_amount):
  
  col=' Source IP'
  value=df[col].iloc[cur_index]
  IP_source_occurrences.append(1)
  for x in range(cur_index-1, max(cur_index-lookback_amount, -1), -1):
    if df[col].iloc[x]==value:
      IP_source_occurrences[cur_index]+=IP_source_occurrences[x]
      break


  col=' Destination IP'
  value=df[col].iloc[cur_index]
  IP_destination_occurrences.append(1)
  for x in range(cur_index-1, max(cur_index-lookback_amount, -1), -1):
    if df[col].iloc[x]==value:
      IP_destination_occurrences[cur_index]+=IP_destination_occurrences[x]
      break

In [ ]:
IP_source_occurrences=[1]
IP_destination_occurrences=[1]

for x in range(1, df.shape[0]):
  count_occurrences(cur_index=x, lookback_amount=100)
df['Source_IP_Occurrences']=IP_source_occurrences
df['Destination_IP_Occurrences']=IP_destination_occurrences


IP_source_occurrences=[1]
IP_destination_occurrences=[1]

for x in range(1, df.shape[0]):
  count_occurrences(cur_index=x, lookback_amount=3000)
df['Source_IP_Occurrences_1000']=IP_source_occurrences
df['Destination_IP_Occurrences_1000']=IP_destination_occurrences

In [ ]:
df.drop([' Source IP',' Destination IP'],axis=1,inplace=True)
df

,Source Port,Destination Port,Protocol,Timestamp,Label,Source_IP_Occurrences,Destination_IP_Occurrences,Source_IP_Occurrences_1000,Destination_IP_Occurrences_1000
0,443,58894,6,100,BENIGN,1,1,1,1
1,443,51906,6,100,BENIGN,1,1,1,1
2,51942,443,6,100,BENIGN,1,1,1,1
3,443,51902,6,100,BENIGN,1,2,1,2
4,35235,53,17,100,BENIGN,1,1,1,1
...,...,...,...,...,...,...,...,...,...
692623,30134,53,17,1259,BENIGN,639,19479,5243,26489
692624,60478,443,6,1259,BENIGN,640,2,5244,24
692625,80,38098,6,1259,BENIGN,5,153,15,1380
692626,39020,443,6,1259,BENIGN,641,1,5245,3


In [ ]:
df.to_csv('export_ddos.csv',index=False)

In [ ]:
df=pd.read_csv('export_ddos.csv')
df

,Source Port,Destination Port,Protocol,Timestamp,Label,Source_IP_Occurrences,Destination_IP_Occurrences,Source_IP_Occurrences_1000,Destination_IP_Occurrences_1000
0,443,58894,6,100,BENIGN,1,1,1,1
1,443,51906,6,100,BENIGN,1,1,1,1
2,51942,443,6,100,BENIGN,1,1,1,1
3,443,51902,6,100,BENIGN,1,2,1,2
4,35235,53,17,100,BENIGN,1,1,1,1
...,...,...,...,...,...,...,...,...,...
692623,30134,53,17,1259,BENIGN,639,19479,5243,26489
692624,60478,443,6,1259,BENIGN,640,2,5244,24
692625,80,38098,6,1259,BENIGN,5,153,15,1380
692626,39020,443,6,1259,BENIGN,641,1,5245,3


In [ ]:
input_data = df.drop(' Label',axis=1)

In [ ]:
output_data = []
for x in range(df.shape[0]):
			if df[' Label'].iloc[x]=="BENIGN":
				output_data.append(0)
			else:
				output_data.append(1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc = MinMaxScaler()
input_data=sc.fit_transform(input_data)

In [ ]:
train_size = 0.7
X_train = np.array(input_data[ : int(len(input_data)*train_size)])
y_train = np.array(output_data[ : int(len(output_data)*train_size)])
X_test = np.array(input_data[int(len(input_data)*train_size) : ])
y_test = np.array(output_data[int(len(output_data)*train_size) : ])

##Neural Network

In [ ]:
def train_model(X_train, y_train):
  model = Sequential()
  model.add(Dense(input_dim = len(X_train[0]), units = int(len(X_train)/1), activation = 'relu'))
  model.add(Dropout(rate = 0.3))
  model.add(Dense(units = int(len(X_train[0])), activation = 'relu'))
  model.add(Dropout(rate = 0.3))
  model.add(Dense(units = 1, activation = 'sigmoid'))

  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  model.fit(X_train, y_train, batch_size = 20, epochs = 20)

  return model

In [ ]:
model=train_model(X_train,y_train)

Epoch 1/20
20742/20742 [==============================] - 536s 26ms/step - loss: 0.0328 - accuracy: 0.9907
Epoch 2/20
20742/20742 [==============================] - 530s 26ms/step - loss: 0.0107 - accuracy: 0.9978
Epoch 3/20
20742/20742 [==============================] - 526s 25ms/step - loss: 0.0105 - accuracy: 0.9978
Epoch 4/20
20742/20742 [==============================] - 521s 25ms/step - loss: 0.0092 - accuracy: 0.9982
Epoch 5/20
20742/20742 [==============================] - 520s 25ms/step - loss: 0.0092 - accuracy: 0.9982
Epoch 6/20
20742/20742 [==============================] - 524s 25ms/step - loss: 0.0093 - accuracy: 0.9982
Epoch 7/20
20742/20742 [==============================] - 526s 25ms/step - loss: 0.0091 - accuracy: 0.9983
Epoch 8/20
20742/20742 [==============================] - 524s 25ms/step - loss: 0.0087 - accuracy: 0.9984
Epoch 9/20
20742/20742 [==============================] - 523s 25ms/step - loss: 0.0089 - accuracy: 0.9984
Epoch 10/20
20742/20742 [============

In [ ]:
y_pred = model.predict(X_test)
print(np.unique(y_pred))
y_pred = (y_pred > 0.005)

[0.0000000e+00 1.1778247e-38 1.1789935e-38 ... 9.9999982e-01 9.9999988e-01
 1.0000000e+00]


In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix: ")
print(str(cm))

if len(cm) == 1:
  print("Undesirable confusion matrix, the neural network predicted on a single class for all data points. Try training with a better configuration, or testing with more occurances of both target classes.")

Confusion matrix: 
[[66095 17976]
 [10291 83427]]


In [ ]:
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

In [ ]:
print("True Positive Rate: {}".format(TP/(TP+FP)))
print("False Negative Rate: {}".format(FN/(TN+FN)))

True Positive Rate: 0.8227271382503476
False Negative Rate: 0.1347236404576755


In [ ]:
model.evaluate(X_test,y_test)*100

5556/5556 [==============================] - 43s 8ms/step - loss: 16.9095 - accuracy: 0.8529


[16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.90950584411621,
 0.8529492616653442,
 16.909505844116

In [ ]:
model.save("DDoSDetector")

INFO:tensorflow:Assets written to: DDoSDetector/assets


In [ ]:
model = tf.keras.models.load_model("DDoSDetector")

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train,y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [ ]:
y_pred = rf.predict(X_test)
y_pred = (y_pred > 0.005)

In [ ]:
accuracy_score(y_pred,y_test)

0.9968910770060013

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 85898,    646],
       [     0, 121245]])

In [ ]:
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

print("True Positive Rate: {}".format(TP/(TP+FP)))
print("False Negative Rate: {}".format(FN/(TN+FN)))

True Positive Rate: 0.9947001829503409
False Negative Rate: 0.0


##LightGBM

In [ ]:
import lightgbm
lgbm = lightgbm.LGBMClassifier(colsample_bytree=0.7)
lgbm.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.7)

In [ ]:
y_pred=lgbm.predict(X_test)

In [ ]:
accuracy_score(y_pred,y_test)

0.9997786215824707

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 86544,      0],
       [    46, 121199]])

In [ ]:
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

print("True Positive Rate: {}".format(TP/(TP+FP)))
print("False Negative Rate: {}".format(FN/(TN+FN)))

True Positive Rate: 1.0
False Negative Rate: 0.0005312391731146784
